<a href="https://colab.research.google.com/github/shrinet/DeepLearning-Study/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np

In [ ]:
torch.__version__

'2.4.1+cu121'

In [ ]:
torch.manual_seed(42)

## Q1. Load the California Housing dataset from sklearn. How many features are there in the dataset?

In [ ]:
from sklearn.datasets import fetch_california_housing
train, y = fetch_california_housing(return_X_y=True, as_frame=True)
train.shape, y.shape

((20640, 8), (20640,))

In [ ]:
train

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [ ]:
y

,MedHouseVal
0,4.526
1,3.585
2,3.521
3,3.413
4,3.422
...,...
20635,0.781
20636,0.771
20637,0.923
20638,0.847


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import pandas as pd
pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7
0,-1.155085,-0.286324,-0.520686,-0.171746,-0.030301,0.067408,0.195100,0.285347
1,-0.708659,0.110435,-0.165815,0.223472,0.121851,-0.036030,-0.235491,0.060975
2,-0.210402,1.856173,-0.610765,0.205892,-0.102419,-0.149989,1.009478,-1.424870
3,0.975113,-0.921138,0.305115,-0.176552,0.244979,0.030011,-0.637999,0.429943
4,-0.081794,0.427842,0.024193,-0.158465,-0.319653,-0.052971,0.457199,-1.170581
...,...,...,...,...,...,...,...,...
4123,0.380334,-1.000489,0.656380,-0.069975,-0.066360,-0.009330,-1.068590,1.177851
4124,-0.606098,-0.048268,0.291827,0.369465,0.196607,-0.065340,-0.132523,-0.622115
4125,2.809024,-0.286324,0.755012,-0.345109,0.139441,-0.026469,0.780141,-1.230414
4126,-0.575430,0.586545,-0.061243,-0.262147,-0.175417,-0.043905,0.527404,-0.088607


### Q2. Split the data into training and test sets while keeping 20% of the data for testing.Keep the random state to be 42.
### Use StandardScaler to standardize the features.

### What are the minimum and maximum feature values (rounded) of the last test data point?

In [ ]:
max(X_test[4127]), min(X_test[4127])

(0.6044549279679837, -0.9211376319710947)

## Q3. Convert the train and test data into PyTorch tensors with the data type to be torch.float32. Reshape the target vectors of train and test data to (-1, 1).
### What are the shapes of train data and train label tensors?

In [ ]:
tensor = torch.from_numpy(X_train).type(torch.float32)
target_tensor = torch.from_numpy(y_train.values).type(torch.float32).reshape(-1, 1)
tensor.shape, target_tensor.shape

(torch.Size([16512, 8]), torch.Size([16512, 1]))

## Q4. Define a Neural Network class. Let's name it RegressionANN. It inherits from nn.Module, which is a base class for all neural network modules in PyTorch.
### The network should have an input layer, one hidden layer, and an output layer.
### To handle California Housing dataset, which has 8 features, the input layer should have 8 neurons (one for each feature in the dataset) (Hint: Use nn.Linear())
### The hidden layer should have 16 neurons. (We chose 16 arbitrarily; in practice, you might tune this number.)
### The output layer should have 1 neuron since we're predicting a single value (the median house value).
### In the class, define a forward method that defines how the data flows through the network. Apply a ReLU activation function after the hidden layer to introduce non-linearity. Create an instance of this RegressionANN class.

### What is the value of the (initial) bias in the output layer?

In [ ]:
import torch
import torch.nn as nn

class RegressionANN(nn.Module):
    def __init__(self):
        super(RegressionANN, self).__init__()
        # Input layer with 8 neurons, hidden layer with 16 neurons, output layer with 1 neuron
        self.input_layer = nn.Linear(8, 16)
        self.hidden_layer = nn.Linear(16, 1)

    def forward(self, x):
        # Forward pass through the network
        x = self.input_layer(x)
        x = torch.relu(x)  # ReLU activation after hidden layer
        x = self.hidden_layer(x)
        return x

# Create an instance of the RegressionANN class
model = RegressionANN()

# Check the initial bias value in the output layer
initial_output_bias = model.hidden_layer.bias.item()
print("Initial bias in the output layer:", initial_output_bias)


Initial bias in the output layer: 0.22723788022994995


## Q5.

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

output = model(tensor)
initial_loss = criterion(output, target_tensor)
print("initial loss",initial_loss.item())

initial loss 4.202902793884277


## Q6.

In [ ]:
no_epoc = 100
for epoch in range(no_epoc):
    optimizer.zero_grad()
    output = model(tensor)
    loss = criterion(output, target_tensor)
    loss.backward()
    optimizer.step()

print("Final loss after 100 iteration:", loss.item())

Final loss after 100 iteration: 0.4749816060066223


## Q7. With 64 hidden neuron




In [ ]:
import torch
import torch.nn as nn

class RegressionANN(nn.Module):
    def __init__(self):
        super(RegressionANN, self).__init__()
        # Input layer with 8 neurons, hidden layer with 16 neurons, output layer with 1 neuron
        self.input_layer = nn.Linear(8, 64)
        self.hidden_layer = nn.Linear(64, 1)

    def forward(self, x):
        # Forward pass through the network
        x = self.input_layer(x)
        x = torch.relu(x)  # ReLU activation after hidden layer
        x = self.hidden_layer(x)
        return x

model2 = RegressionANN()

In [ ]:
no_epoc = 100

criterion = nn.MSELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.01)

for epoch in range(no_epoc):
    optimizer.zero_grad()
    output = model2(tensor)
    loss = criterion(output, target_tensor)
    loss.backward()
    optimizer.step()

print("Final loss after 100 iteration:", loss.item())

Final loss after 100 iteration: 0.41575008630752563
